## Import Needed Libraries

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import logging
import warnings
import os
from pandas import ExcelWriter
import ast
import math

# ML Training 
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler

# Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

# hide warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

# General settings
pd.set_option('display.max_colwidth', None)
np.random.seed(11)
nltk.download('wordnet')

domain = "Social_Networking"
#domain = "Games"
#domain = "Productivity"

text_representation_url = ""
all_features_url = ""

if domain == "Social_Networking":
    text_representation_url = "https://drive.google.com/uc?id=1-obCouGNTpXLS0UPbwazeuZMNRQCpO_V&export=download"
    all_features_url = "https://drive.google.com/uc?id=/1QGRqvdnVUst9BQh3_KetuRcrRxWZZSxR&export=download"
elif domain == "Games":
    text_representation_url = "https://drive.google.com/uc?id=1mh6gURnnFJ-KeBRpmF9gHjtGhoQ2D6OI&export=download"
    all_features_url = "https://drive.google.com/uc?id=15zkGkJe5SXA6jEaxUiUE5OQyOC0A7uHg&export=download"
elif domain == "Productivity":
    text_representation_url = "https://drive.google.com/uc?id=1AT7-wtIblIlY6HZYzwyQZy_cQWoh8pvE&export=download"
    all_features_url = "https://drive.google.com/uc?id=0t2J8cXcII_4VgGzcvFgSPTD0PekqcRC&export=download"

# Load data

In [ ]:
# Read data from csv

df_1 = pd.read_csv(text_representation_url)

df_1['tfidf'] = [np.array(ast.literal_eval(x)) for x in df_1['tfidf']]
df_1['w2v'] = [np.array(ast.literal_eval(x)) for x in df_1['w2v']]
df_1['ds_w2v'] = [np.array(ast.literal_eval(x)) for x in df_1['ds_w2v']]
df_1['w2v_tfidf'] = [np.array(ast.literal_eval(x)) for x in df_1['w2v_tfidf']]
df_1['ds_w2v_tfidf'] = [np.array(ast.literal_eval(x)) for x in df_1['ds_w2v_tfidf']]
df_1['elmo'] = [np.array(ast.literal_eval(x)) for x in df_1['elmo']]
df_1['bert'] = [np.array(ast.literal_eval(x)) for x in df_1['bert']]
df_1['fine_tuned_bert'] = [np.array(ast.literal_eval(x)) for x in df_1['fine_tuned_bert']]

In [ ]:
# All features
df_2 = pd.read_csv(all_features_url)
df_2.drop(['sentence', 'category', 'category_id'], axis='columns', inplace=True)

df = pd.concat([df_1,df_2], axis=1)

## Training

In [ ]:
# print columns (features) names
df.columns

In [ ]:
# select features for the ablation study 

#df_features_1 = df.iloc[:, 16:32]
#df_features_2 = df.iloc[:, 40:48]
#df_features = pd.concat([df_features_1,df_features_2], axis=1)

df_features = df.iloc[:, 16:]
df_features.columns

In [ ]:
%%time

# open excel file to export the results to it
writer = ExcelWriter("Results/ACD/" + domain + "_all.xlsx")
    
classifiers = {"SVM": LinearSVC(max_iter=100, random_state=11),
               "MLP": MLPClassifier(max_iter=100, random_state=11),
               "DT": DecisionTreeClassifier(random_state=11),
               "GNB": GaussianNB(),
               "LR": LogisticRegression(random_state=11),
               "KNN": KNeighborsClassifier()
              }

text_representations = ["tfidf", "w2v", "ds_w2v", "w2v_tfidf", "ds_w2v_tfidf", "elmo", "bert", "fine_tuned_bert"]

scoring = ['f1_micro']

col_names = ['model']
for x in text_representations:
    col_names.append(x)
output_results = []

for name, classifier in classifiers.items():

    avereged_results = [name]
    
    for text_representation in text_representations:
        
        np.random.seed(11)

        print("\ncreate the pipeline for " + name + " and " +  text_representation)
                
        pipeline = Pipeline([
            ('smote', SMOTE()),
            ('cls', classifier)
        ])
    
    
        X = np.concatenate((np.array(df[text_representation].tolist()), df_features.values), axis=1)
        #X = np.array(df[text_representation].tolist())
        Y = df["category_id"]
        
        srkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=11)
        results = cross_validate(pipeline, X, Y, cv=srkf, scoring=scoring) 
        
        f1 = np.mean(results['test_f1_micro']) * 100
        print('Micro F1-Score:', f1, '%')
        avereged_results.append(f1)
    output_results.append(avereged_results)
    

# export classifier's results
result_df = pd.DataFrame(output_results, columns=col_names)
result_df.to_excel(writer, index=False)


writer.save()
os.system("printf '\a'") # or '\7'